In [1]:
# Instalando a biblioteca "scikit-surprise" no ambiente Python.
!pip install scikit-surprise

In [2]:
import heapq # Importando o módulo "heapq", que fornece funções relacionadas a filas de prioridade (heap).
import os # Importando o módulo "os", que fornece funcionalidades para interagir com o sistema operacional.
import csv # Importando o módulo "csv", que fornece funcionalidades para trabalhar com arquivos CSV (Comma-Separated Values).
from surprise import KNNBasic # Importando a classe "KNNBasic" usada para criar um modelo de filtro colaborativo para sistemas de recomendação.
from surprise import Dataset # Importando a classe "Dataset" que é usada para carregar e manipular conjuntos de dados usados em sistemas de recomendação. 
from surprise import Reader # Importando a classe "Reader" que é usada para definir a forma como os dados são lidos e interpretados.
from collections import defaultdict # Importando a classe "defaultdict", sendo um dicionário em que pode-se definir um valor padrão para chaves não existentes.
from operator import itemgetter # Importando a função "itemgetter", usada para acessar elementos de um objeto iterável, como uma lista, com base em índices ou chaves. 

In [3]:
# Carregando as classificações dos filmes e retornando um conjunto de dados.
def load_dataset():
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    ratings_dataset = Dataset.load_from_file('MovieLens/ratings.csv', reader=reader)
# O pacote de dados do MovieLens, utilizado nesse exemplo, pode ser encontrado em: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

# Busca o nome de um filme com o ID do Movielens como chave
    movieID_to_name = {}
    with open('MovieLens/movies.csv', newline='', encoding='ISO-8859-1') as csvfile:
            movie_reader = csv.reader(csvfile)
            next(movie_reader)
            for row in movie_reader:
                movieID = int(row[0])
                movie_name = row[1]
                movieID_to_name[movieID] = movie_name
    # Retorna tanto o conjunto de dados quanto o dicionário de consulta em uma tupla (Lista imutável)
    return (ratings_dataset, movieID_to_name)

In [4]:
# Carregando os dados de avaliações e mapeando os IDs de filmes para seus nomes
dataset, movieID_to_name = load_dataset()

In [5]:
# Construindo um conjunto de treinamento completo do Surprise a partir do conjunto de dados recebidos anteriormente.
trainset = dataset.build_full_trainset()

In [6]:
# Calculando uma matriz de similaridade dos filmes usando k-Nearest Neighbors (KNN) com base em filtragem colaborativa.
# A matriz final representa o quão semelhantes são os itens uns aos outros com base nas avaliações dos usuários.
similarity_matrix = KNNBasic(sim_options={
        'name': 'cosine',
        'user_based': False
        })\
        .fit(trainset)\
        .compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [7]:
# Escolha um ID de usuário e veja como as recomendações finais, IDs númericos entre 1 e 610;

test_subject = '1' # ID de um usuário escolhido

k = 20 # Número de avaliações do usuário informado acima para utilização como base

In [8]:
# Identificando os itens mais bem avaliados por um usuário específico no conjunto de treinamento e os armazena na lista k_neighbors

# Convertendo o ID do usuário (que pode ser uma string ou número) em um ID exclusivo que a biblioteca Surprise pode usar mais facilmente para cálculos internos.
# A ideia é que a biblioteca trabalha de maneira mais eficiente com IDs inteiros exclusivos.
test_subject_iid = trainset.to_inner_uid(test_subject)

# Buscando os principais K itens avaliados
test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbors = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])

In [9]:
# Construindo uma lista de candidatos para recomendação de filmes com base em filmes similares aos que o usuário já classificou
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue

In [10]:
# Função usada para obter o nome de um filme com base em seu ID. caso o ID não exista, retorna uma string vazia.
def getMovieName(movieID):
  if int(movieID) in movieID_to_name:
    return movieID_to_name[int(movieID)]
  else:
      return ""

In [11]:
# Construindo um dicionário com os filmes que o usuário já assistiu.
watched = {} # Inicia um dicionario vazio
for itemID, rating in trainset.ur[test_subject_iid]:
  watched[itemID] = 1
# Adiciona o ID do filme como uma chave no dicionário com valor 1, indicando que o filme já foi assistido

In [12]:
# Gerando uma lista de filmes com base nas informações dos candidatos

recommendations = [] #Lista inicialmente vazia
position = 0 # Variável  para rastrear a posição dos filmes recomendados na lista.

# Loop sobre os itens candidatos a recomendação com similaridades ponderadas
# A função sorted ordena os itens em ordem decrescente de classificação (filmes mais relevantes aparecem primeiro)
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in watched: #Verifica se o filme já foi assistido pelo usuário
    recommendations.append(getMovieName(trainset.to_raw_iid(itemID)))
    position += 1
    if (position > 4): break # Limita o numero de filmes que serão selecionados e gravados na lista 
    
# Loop iterando sobre a lista de recomendações e imprime o nome de cada filme recomendado.
for i, rec in enumerate(recommendations, start=1):
  print(f"{i} - Filme: ", rec) 

1 - Filme:  Hang 'Em High (1968)
2 - Filme:  Moonlight Mile (2002)
3 - Filme:  Meet John Doe (1941)
4 - Filme:  Audition (Ãdishon) (1999)
5 - Filme:  Other Boleyn Girl, The (2008)


In [13]:
# Gerando uma lista de filmes com base nas informações dos candidatos
def get_movie_id_by_name(movie_name, movieID_to_name):
    for movie_id, name in movieID_to_name.items():
        if movie_name.lower() in name.lower():
            return movie_id
    return None

In [14]:
### Buscando um filme e recebendo recomendações com base nas avaliações de outros usuários e filmes ###

# Nome do filme fornecido pelo usuário
user_movie_name = "Top Gun" 

# Encontrando o ID do filme com base no nome
user_movie_id = get_movie_id_by_name(user_movie_name, movieID_to_name)

if user_movie_id is not None:
    # Calculando a similaridade entre o filme fornecido e todos os outros filmes
    similarities = similarity_matrix[user_movie_id]

    # Criando uma lista de filmes recomendados com base na similaridade
    recommended_movies = []
    for movie_id, similarity_score in enumerate(similarities):
        if movie_id != user_movie_id:
            recommended_movies.append((movie_id, similarity_score))

    # Ordenando a lista de recomendações com base na similaridade
    recommended_movies.sort(key=lambda x: x[1], reverse=True)

    # Imprimir os filmes recomendados
    top_n = 5  # Número de filmes a recomendar
    print(f"Recomendações para o filme '{user_movie_name}':")
    for position, (movie_id, similarity_score) in enumerate (recommended_movies[:top_n], start = 1):
        recommended_movie_name = movieID_to_name.get(movie_id, "Desconhecido") #Retorna desconhecido caso não encontre a ID do filme
        print(f"{position} - Filme: {recommended_movie_name} ") #(Similaridade: {similarity_score:.2f}))
else:
    print(f"O filme '{user_movie_name}' não foi encontrado no conjunto de dados.")


Recomendações para o filme 'Top Gun':
1 - Filme: Dolores Claiborne (1995) 
2 - Filme: Mixed Nuts (1994) 
3 - Filme: Miami Rhapsody (1995) 
4 - Filme: Murder in the First (1995) 
5 - Filme: Nina Takes a Lover (1994) 


In [15]:
print("\nXVI Fórum Científico | FEMA | PIC Rubens Junior, Guilherme Farto")
print("\n\tImplementação de um algoritmo de recomendação utilizando a biblioteca Sklearn-Surprise e a base de dados da MovieLens.")


XVI Fórum Científico | FEMA | PIC Rubens Junior, Guilherme Farto

	Implementação de um algoritmo de recomendação utilizando a biblioteca Sklearn-Surprise e a base de dados da MovieLens.
